In [3]:
import pandas as pd
import datetime
import seaborn as sns
from scipy import stats
import numpy as np
import random

In [1]:
filename = "./../data/311_Cases.csv"

In [4]:
data = pd.read_csv(filename, encoding='utf8',engine='python').dropna()

In [8]:
# quick inspection
print(len(data)) # 539,576 (after dropping NAs)
data.head()

539576


,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
24810,3135216,12/05/2013 02:50:24 PM,06/25/2018 09:50:55 AM,06/25/2018 09:50:55 AM,Closed,Found 40 square feet of damaged/uneven sidewal...,DPW BSM Queue,Sidewalk or Curb,Sidewalk_Defect,Collapsed_sidewalk,"920 VAN NESS AVE, SAN FRANCISCO, CA, 94109",6.0,Tenderloin,NORTHERN,37.78418,-122.4208,"(37.78416124, -122.42081673)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...
86201,4203877,11/12/2014 08:46:11 PM,06/25/2018 10:38:19 AM,06/25/2018 10:38:19 AM,Closed,9 Sumner St: damaged sidewalk around utility v...,DPW BSM Queue,Sidewalk or Curb,Sidewalk_Defect,Collapsed_sidewalk,"10 SUMNER ST, SAN FRANCISCO, CA, 94103",6.0,South of Market,SOUTHERN,37.77632,-122.4106,"(37.77629119, -122.41061191)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...
86582,4223752,11/19/2014 10:48:56 AM,06/25/2018 10:22:00 AM,06/25/2018 10:22:00 AM,Closed,2515-2565 Bay Shore Blvd: damaged sidewalk nex...,DPW BSM Queue,Sidewalk or Curb,Sidewalk_Defect,Collapsed_sidewalk,Intersection of BAY SHORE BLVD and SUNNYDALE AVE,10.0,Little Hollywood,INGLESIDE,37.70903,-122.4048,"(37.70885355, -122.4051094)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...
100891,10303151,01/01/2019 10:10:00 PM,01/08/2019 05:46:00 AM,01/08/2019 05:46:00 AM,Closed,01/07/2019 VEH GOA @1033 HOURS #338,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicles,DPT Abandoned Vehicles Low,"1097 OAKDALE AVE, SAN FRANCISCO, CA, 94124",10.0,Hunters Point,BAYVIEW,37.72940,-122.3813,"(37.72939344, -122.38131978)",Web,http://mobile311.sfgov.org/media/san_francisco...
140933,10303744,01/02/2019 08:20:00 AM,01/08/2019 07:39:00 AM,01/08/2019 07:39:00 AM,Closed,01/07/2019 @1349 HRS #168,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicles,DPT Abandoned Vehicles Low,Intersection of MALTA DR and VALLETTA CT,7.0,Miraloma Park,INGLESIDE,37.73619,-122.4444,"(37.7362185, -122.4444058)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...


In [9]:
# set random seed
random.seed(42)
small_data = data.sample(n = int(len(data) * 0.2)) # keep 20%
len(small_data) # 107,915

107915

# Select relevant variables

In [17]:
# create ClosingTime (target variable)
timedelta = pd.to_datetime(small_data["Closed"]) - pd.to_datetime(small_data["Opened"])

In [49]:
small_data['Closing Time'] = timedelta.dt.days
for_csv =  small_data[['Closing Time', "Neighborhood", "Category", 'Police District', 'Responsible Agency']]

# filter negatives
for_csv[for_csv['Closing Time'] > 0].sort_values(by=['Closing Time'])

,Closing Time,Neighborhood,Category,Police District,Responsible Agency
2804469,1,North Beach,Street and Sidewalk Cleaning,CENTRAL,RPD Park Service Area 1 Queue
1699311,1,Potrero Hill,Streetlights,MISSION,PUC Streetlights Queue
1970776,1,Chinatown,Graffiti,CENTRAL,DPW Ops Queue
1454619,1,Panhandle,Graffiti,PARK,DPW Ops Queue
1191996,1,Inner Richmond,Graffiti,RICHMOND,DPW Ops Queue
2606666,1,North Beach,Graffiti,CENTRAL,DPT Meter_Bike Queue
2140623,1,Mission,Street and Sidewalk Cleaning,MISSION,DPW Ops Queue
1786287,1,Russian Hill,Graffiti,CENTRAL,DPW Ops Queue
3291428,1,South Beach,Encampments,SOUTHERN,HSOC Queue
2890857,1,South of Market,General Request - PUBLIC WORKS,SOUTHERN,HSOC Queue


# Save to CSV

In [50]:
for_csv.to_csv("./../data/311_Cases_small.csv", header=False, index=False)